In [2]:
from transformers import pipeline
from youtube_transcript_api import YouTubeTranscriptApi
from deepface import DeepFace





/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


25-06-23 00:02:15 - Directory /root/.deepface has been created
25-06-23 00:02:15 - Directory /root/.deepface/weights has been created


In [3]:
def get_youtube_id(url):
    match = re.search(r"(?<=v=|\/shorts\/)[^&]+", url)
    return match.group(0) if match else None



In [4]:
def fetch_transcript(video_id):
    try:
        return YouTubeTranscriptApi.get_transcript(video_id, languages=['es', 'en'])
    except:
        return []


In [5]:
sentiment_model = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

def analyze_sentiment(text_list):
    full_text = " ".join([x['text'] for x in text_list])
    return sentiment_model(full_text[:512])

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
def get_thumbnail_emotions(thumbnail_url):
    response = requests.get(thumbnail_url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    img.save("thumbnail.jpg")

    emotion = DeepFace.analyze(img_path="thumbnail.jpg", actions=["emotion"], enforce_detection=False)
    return emotion[0]["emotion"]


In [7]:
def get_thumbnail_colors(thumbnail_url):
    response = requests.get(thumbnail_url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    img.save("thumb_color.jpg")
    colors = colorgram.extract("thumb_color.jpg", 5)
    return [{"rgb": c.rgb, "proportion": c.proportion} for c in colors]


In [8]:
def generate_recommendations(emo_result, sentiment_result):
    recs = []
    dominant_emotion = max(emo_result, key=emo_result.get)
    avg_star = int(sentiment_result[0]['label'][0])

    if dominant_emotion in ['sad', 'angry'] and avg_star < 3:
        recs.append("Prueba contenido más positivo o visualmente colorido.")
    elif dominant_emotion in ['happy', 'surprise']:
        recs.append("Amplifica ese tono con música energética y hashtags relevantes.")
    if avg_star >= 4:
        recs.append("Buen sentimiento: refuerza visual con miniaturas vibrantes.")
    return recs


In [9]:
video_url = input("🔗 Pega el link del video de YouTube Shorts: ")
video_id = get_youtube_id(video_url)
transcript = fetch_transcript(video_id)
sentiment = analyze_sentiment(transcript)
thumbnail_url = f"https://img.youtube.com/vi/{video_id}/maxresdefault.jpg"
emotions = get_thumbnail_emotions(thumbnail_url)
colors = get_thumbnail_colors(thumbnail_url)
recommendations = generate_recommendations(emotions, sentiment)

output = {
    "video_id": video_id,
    "sentiment": sentiment,
    "emotions": emotions,
    "colors": colors,
    "recommendations": recommendations
}

print(json.dumps(output, indent=4, ensure_ascii=False))


KeyboardInterrupt: Interrupted by user